In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='70760d0aaec6d9204de7a53e371400e9123b1f6e', drake_version='0.27.0', drake_build='release')

from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(
    server_args=['--ngrok_http_tunnel'] if 'google.colab' in sys.modules else [])

from pydrake.common import set_log_level
set_log_level('off');

from underactuated import FindResource
from underactuated.jupyter import running_as_notebook
from pydrake.all import DiagramBuilder, AddMultibodyPlantSceneGraph, MeshcatVisualizer
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import numpy as np
from pydrake.all import AddMultibodyPlantSceneGraph, DiagramBuilder, Parser, ConnectMeshcatVisualizer, RigidTransform, Simulator, PidController
from pydrake.all import MathematicalProgram, OsqpSolver, eq, le, ge
from pydrake.solvers import branch_and_bound

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Insert cd to models folder here

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

def set_home(plant, context, knee = 0):
    starting_height = .1 #.1 should be exactly on ground

    plant.GetJointByName("front_right_knee").set_angle(context, -knee)
    
    plant.GetJointByName("back_right_knee").set_angle(context, knee)
    #plant.GetJointByName("planar_joint").set_translation(context, np.array([0, starting_height]))
    plant.SetFreeBodyPose(context, plant.GetBodyByName("body"), RigidTransform([0, 0, starting_height]))

def run_pid_control(knee = 0):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-5)
    parser = Parser(plant)
    parser.AddModelFromFile('littledog/LittleDog3.urdf')
    parser.AddModelFromFile('littledog/ground.urdf')

    

    plant.Finalize()
    
    # Add a PD Controller
    visualizer = ConnectMeshcatVisualizer(builder, scene_graph=scene_graph, zmq_url=zmq_url)
    visualizer.load()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    set_home(plant, plant_context, knee=knee)
    foot_frame = [
        plant.GetFrameByName('front_right_foot_center'),
        plant.GetFrameByName('back_right_foot_center')]
    bf = plant.GetFrameByName('body')
    for i in foot_frame:
        tf = plant.CalcRelativeTransform(plant_context, bf, i)
        #print(tf.rotation().matrix())
        print(tf.translation())
    COM = plant.CalcCenterOfMassPositionInWorld(plant_context)
    body_pos = plant.CalcPointsPositions(plant_context, bf, [0,0,0], plant.world_frame())
    print(COM)
    print(body_pos)

    sliders = MakeJointSlidersThatPublishOnCallback(plant, visualizer, context)

run_pid_control(knee=0)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://371807718113.ngrok.io/static/
Connected to meshcat-server.
[ 0.0745  0.     -0.0985]
[-0.0745  0.     -0.0985]
[0.         0.         0.06478341]
[[0. ]
 [0. ]
 [0.1]]


FloatSlider(value=0.0, description='front_right_knee', layout=Layout(width="'200'"), max=1.0, min=-3.1, step=0…

FloatSlider(value=0.0, description='back_right_knee', layout=Layout(width="'200'"), max=3.1, min=-1.0, step=0.…

In [ ]:
from functools import partial
from pydrake.all import (
    MultibodyPlant, JointIndex, RotationMatrix, PiecewisePolynomial, JacobianWrtVariable,
    MathematicalProgram, Solve, eq, AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
)
from pydrake.common.containers import namedview

# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(autoDiffToGradientMatrix(a), autoDiffToGradientMatrix(b))

# TODO: promote this to drake (and make a version with model_instance)
def MakeNamedViewPositions(mbp, view_name):
    names = [None]*mbp.num_positions()
    for ind in range(mbp.num_joints()): 
        joint = mbp.get_joint(JointIndex(ind))
        # TODO: Handle planar joints, etc.
        assert(joint.num_positions() == 1)
        names[joint.position_start()] = joint.name()
    for ind in mbp.GetFloatingBaseBodies():
        body = mbp.get_body(ind)
        start = body.floating_positions_start()
        body_name = body.name()
        names[start] = body_name+'_qw'
        names[start+1] = body_name+'_qx'
        names[start+2] = body_name+'_qy'
        names[start+3] = body_name+'_qz'
        names[start+4] = body_name+'_x'
        names[start+5] = body_name+'_y'
        names[start+6] = body_name+'_z'
    return namedview(view_name, names)

def MakeNamedViewVelocities(mbp, view_name):
    names = [None]*mbp.num_velocities()
    for ind in range(mbp.num_joints()): 
        joint = mbp.get_joint(JointIndex(ind))
        # TODO: Handle planar joints, etc.
        assert(joint.num_velocities() == 1)
        names[joint.velocity_start()] = joint.name()
    for ind in mbp.GetFloatingBaseBodies():
        body = mbp.get_body(ind)
        start = body.floating_velocities_start() - mbp.num_positions()
        body_name = body.name()
        names[start] = body_name+'_wx'
        names[start+1] = body_name+'_wy'
        names[start+2] = body_name+'_wz'
        names[start+3] = body_name+'_vx'
        names[start+4] = body_name+'_vy'
        names[start+5] = body_name+'_vz'
    return namedview(view_name, names)

def set_home(plant, context):
    knee = 0
    starting_height = .1 #.1 should be exactly on ground

    plant.GetJointByName("front_right_knee").set_angle(context, -knee)
    
    plant.GetJointByName("back_right_knee").set_angle(context, knee)
    #plant.GetJointByName("planar_joint").set_translation(context, np.array([0, starting_height]))
    plant.SetFreeBodyPose(context, plant.GetBodyByName("body"), RigidTransform([0, 0, starting_height]))


def gaitoptimization(gait="walking"):
    # Set up simulated environment
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    littledog = parser.AddModelFromFile('littledog/LittleDog3.urdf')
    plant.Finalize()
    visualizer = ConnectMeshcatVisualizer(builder, 
        scene_graph=scene_graph, 
        zmq_url=zmq_url)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    set_home(plant, plant_context) # Set robot to default position
    visualizer.load()
    diagram.Publish(context)

    q0 = plant.GetPositions(plant_context)
    body_frame = plant.GetFrameByName("body")

    PositionView = MakeNamedViewPositions(plant, "Positions")
    VelocityView = MakeNamedViewVelocities(plant, "Velocities")

    mu = 1 # rubber on rubber

    # Compute
    total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(littledog))
    gravity = plant.gravity_field().gravity_vector()
    
    print("---------------------------------------------------------------")
    print("Starting state:")
    print("[qw, qx, qy, qz, x, y, z, frontjointangle, backjointangle]")
    print("Note: negative angle means tilting the leg forwards")
    print(q0)
    print("Total mass:")
    print(total_mass)
    print("Gravity vector:")
    print(gravity)

    foot_frame = [
        plant.GetFrameByName('front_right_foot_center'),
        plant.GetFrameByName('back_right_foot_center')]

    """
    TODO: Setup Gait Stuff
    """
    if gait == "walking":
        N = 21
        speed = 0.4
        stride_length = 0.2 # Original was .25, but legs are shorter now
        in_stance = np.zeros((2, N))
        in_stance[0, :11] = 1
        in_stance[0, -1] = 1
        in_stance[1, 0] = 1
        in_stance[1, 8:N] = 1
    elif gait == "running":
        N = 21
        speed = 0.5
        stride_length = 0.25
        in_stance = np.zeros((2, N))
        in_stance[0, :8] = 1
        in_stance[0, -1] = 1
        in_stance[1, 0] = 1
        in_stance[1, 13:N] = 1
    else:
        print("Unknown gait")
        return
    
    T = stride_length / speed

    # Initialize Optimization
    prog = MathematicalProgram()

    # Time steps    
    h = prog.NewContinuousVariables(N-1, "h")
    prog.AddBoundingBoxConstraint(0.5*T/N, 2.0*T/N, h) 
    prog.AddLinearConstraint(sum(h) >= .9*T)
    prog.AddLinearConstraint(sum(h) <= 1.1*T)

    
    
    
    
    # Create one context per timestep (to maximize cache hits)
    context = [plant.CreateDefaultContext() for i in range(N)]
    # We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
    ad_plant = plant.ToAutoDiffXd()

    # Joint positions and velocities
    nq = plant.num_positions()
    nv = plant.num_velocities()
    q = prog.NewContinuousVariables(nq, N, "q")
    v = prog.NewContinuousVariables(nv, N, "v")
    q_view = PositionView(q)
    v_view = VelocityView(v)
    q0_view = PositionView(q0)
    
    # Joint costs
    q_cost = PositionView([1]*nq)
    v_cost = VelocityView([1]*nv)
    
    q_cost.body_x = 0
    q_cost.body_y = 0
    q_cost.body_qx = 0
    q_cost.body_qy = 0
    q_cost.body_qz = 0
    q_cost.body_qw = 0

    v_cost.body_vx = 0
    v_cost.body_wx = 0
    v_cost.body_wy = 0
    v_cost.body_wz = 0

    print("nq:",nq) # COM quaternion orientation (4) + COM xyz position (3) + joint angles (2) = 9
    print("nv:",nv) # COM axis-angle angular velocity (3) + COM xyz velocity (3) + joint angle velocity (2) = 8

    # Formulate optimization problem
    for n in range(N):
        # Joint limits
        prog.AddBoundingBoxConstraint(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits(), q[:,n])
        # Joint velocity limits
        prog.AddBoundingBoxConstraint(plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits(), v[:,n])
        # Unit quaternions
        AddUnitQuaternionConstraintOnPlant(plant, q[:,n], prog)
        # Body orientation
        prog.AddConstraint(OrientationConstraint(plant, 
                                                 body_frame, RotationMatrix(),
                                                 plant.world_frame(), RotationMatrix(), 
                                                 0.1, context[n]), q[:,n])
        # Initial guess for all joint angles is the home position
        prog.SetInitialGuess(q[:,n], q0)  # Solvers get stuck if the quaternion is initialized with all zeros.

        # Running costs:
        prog.AddQuadraticErrorCost(np.diag(q_cost), q0, q[:,n]) # Add cost with q_desired = q0 (default)
        prog.AddQuadraticErrorCost(np.diag(v_cost), [0]*nv, v[:,n])
    
    # Make a new autodiff context for this constraint (to maximize cache hits)
    ad_velocity_dynamics_context = [ad_plant.CreateDefaultContext() for i in range(N)]
    
    def velocity_dynamics_constraint(vars, context_index):
        h, q, v, qn = np.split(vars, [1, 1+nq, 1+nq+nv])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_velocity_dynamics_context[context_index])):
                ad_plant.SetPositions(ad_velocity_dynamics_context[context_index], q)
            v_from_qdot = ad_plant.MapQDotToVelocity(ad_velocity_dynamics_context[context_index], (qn - q)/h)
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            v_from_qdot = plant.MapQDotToVelocity(context[context_index], (qn - q)/h)
        return v - v_from_qdot

    # Enforce dynamics: velocity decision variables = actual velocity
    for n in range(N-1):
        prog.AddConstraint(
            partial(velocity_dynamics_constraint, context_index=n), 
            lb=[0]*nv, ub=[0]*nv, 
            vars=np.concatenate(([h[n]], q[:,n], v[:,n], q[:,n+1])))
    
    # Contact forces
    num_feet = 2

    # Enforce dynamics: friction & contact forces
    contact_force = [prog.NewContinuousVariables(3, N-1, f"foot{foot}_contact_force") for foot in range(num_feet)]
    for n in range(N-1):
        for foot in range(num_feet):
            # Linear friction cone
            prog.AddLinearConstraint(contact_force[foot][0,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(-contact_force[foot][0,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(contact_force[foot][1,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(-contact_force[foot][1,n] <= mu*contact_force[foot][2,n])
            # normal force >=0, normal_force == 0 if not in_stance
            # ASSUMES THAT IN_STANCE IS GIVEN
            prog.AddBoundingBoxConstraint(0, in_stance[foot,n]*4*9.81*total_mass, contact_force[foot][2,n])

    # Enforce dynamics: center of mass
    # Center of mass variables and constraints
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N-1, "comddot")
    # Initial CoM x,y position == 0
    prog.AddBoundingBoxConstraint(0, 0, com[:2,0])
    # Initial CoM z vel == 0
    prog.AddBoundingBoxConstraint(0, 0, comdot[2,0])
    # CoM height
    prog.AddBoundingBoxConstraint(.08, np.inf, com[2,:])
    # CoM x velocity >= 0
    prog.AddBoundingBoxConstraint(0, np.inf, comdot[0,:])
    # CoM final x position
    prog.AddBoundingBoxConstraint(stride_length, stride_length, com[0,-1])
    # CoM dynamics
    for n in range(N-1):
        prog.AddConstraint(eq(com[:, n+1], com[:,n] + h[n]*comdot[:,n]))
        prog.AddConstraint(eq(comdot[:, n+1], comdot[:,n] + h[n]*comddot[:,n]))
        prog.AddConstraint(eq(total_mass*comddot[:,n], sum(contact_force[i][:,n] for i in range(num_feet)) + total_mass*gravity))    
    
    # Angular momentum (about the center of mass)
    H = prog.NewContinuousVariables(3, N, "H")
    Hdot = prog.NewContinuousVariables(3, N-1, "Hdot")
    prog.SetInitialGuess(H, np.zeros((3, N)))
    prog.SetInitialGuess(Hdot, np.zeros((3,N-1)))
    # Hdot = sum_i cross(p_FootiW-com, contact_force_i)
    
    def angular_momentum_constraint(vars, context_index):
        q, com, Hdot, contact_force = np.split(vars, [nq, nq+3, nq+6])
        contact_force = contact_force.reshape(3, num_feet, order='F')
        if isinstance(vars[0], AutoDiffXd):
            q = autoDiffToValueMatrix(q)
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_feet):
                p_FW = plant.CalcPointsPositions(context[context_index], foot_frame[i], [0,0,0], plant.world_frame())
                Jq_FW = plant.CalcJacobianTranslationalVelocity(
                    context[context_index], JacobianWrtVariable.kQDot,
                    foot_frame[i], [0, 0, 0], plant.world_frame(), plant.world_frame())
                ad_p_FW = initializeAutoDiffGivenGradientMatrix(p_FW, np.hstack((Jq_FW, np.zeros((3, 18)))))
                torque = torque     + np.cross(ad_p_FW.reshape(3) - com, contact_force[:,i])
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_feet):
                p_FW = plant.CalcPointsPositions(context[context_index], foot_frame[i], [0,0,0], plant.world_frame())
                torque += np.cross(p_FW.reshape(3) - com, contact_force[:,i])
        return Hdot - torque
    
    

    # Enforce dynamics: torque decision variables = actual torque
    for n in range(N-1):
        prog.AddConstraint(eq(H[:,n+1], H[:,n] + h[n]*Hdot[:,n]))
        Fn = np.concatenate([contact_force[i][:,n] for i in range(num_feet)])
        prog.AddConstraint(partial(angular_momentum_constraint, context_index=n), lb=np.zeros(3), ub=np.zeros(3), 
                           vars=np.concatenate((q[:,n], com[:,n], Hdot[:,n], Fn)))
    
    
    # Make a new autodiff context for this constraint (to maximize cache hits)
    com_constraint_context = [ad_plant.CreateDefaultContext() for i in range(N)]
    def com_constraint(vars, context_index):
        qv, com, H = np.split(vars, [nq+nv, nq+nv+3])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(qv, ad_plant.GetPositionsAndVelocities(com_constraint_context[context_index])):
                ad_plant.SetPositionsAndVelocities(com_constraint_context[context_index], qv)
            com_q = ad_plant.CalcCenterOfMassPositionInWorld(com_constraint_context[context_index])
            H_qv = ad_plant.CalcSpatialMomentumInWorldAboutPoint(com_constraint_context[context_index], com).rotational()
        else:
            if not np.array_equal(qv, plant.GetPositionsAndVelocities(context[context_index])):
                plant.SetPositionsAndVelocities(context[context_index], qv)
            com_q = plant.CalcCenterOfMassPositionInWorld(context[context_index])
            H_qv = plant.CalcSpatialMomentumInWorldAboutPoint(context[context_index], com).rotational()
        return np.concatenate((com_q - com, H_qv - H))
    
    for n in range(N):
        tempvars = np.concatenate((q[:,n], v[:,n], com[:,n], H[:,n]))
        prog.AddConstraint(partial(com_constraint, context_index=n), 
            lb=np.zeros(6), ub=np.zeros(6), vars=tempvars)
        
    
         
    # Kinematic constraints
    def fixed_position_constraint(vars, context_index, frame):
        q, qn = np.split(vars, [nq])
        if not np.array_equal(q, plant.GetPositions(context[context_index])):
            plant.SetPositions(context[context_index], q)
        if not np.array_equal(qn, plant.GetPositions(context[context_index+1])):
            plant.SetPositions(context[context_index+1], qn)
        p_AQ = plant.CalcPointsPositions(context[context_index], frame, [0,0,0], plant.world_frame())
        p_AQn = plant.CalcPointsPositions(context[context_index+1], frame, [0,0,0], plant.world_frame())
        if isinstance(vars[0], AutoDiffXd):
            J = plant.CalcJacobianTranslationalVelocity(context[context_index], JacobianWrtVariable.kQDot,
                                                    frame, [0, 0, 0], plant.world_frame(), plant.world_frame())
            Jn = plant.CalcJacobianTranslationalVelocity(context[context_index+1], JacobianWrtVariable.kQDot,
                                                    frame, [0, 0, 0], plant.world_frame(), plant.world_frame())
            return initializeAutoDiffGivenGradientMatrix(
                p_AQn - p_AQ, Jn @ autoDiffToGradientMatrix(qn) - J @ autoDiffToGradientMatrix(q))
        else:
            return p_AQn - p_AQ
    
    for i in range(num_feet):
        for n in range(N):
            if in_stance[i, n]:
                # foot should be on the ground (world position z=0)
                prog.AddConstraint(PositionConstraint(
                    plant, plant.world_frame(), [-np.inf,-np.inf,0], [np.inf,np.inf,0], 
                    foot_frame[i], [0,0,0], context[n]), q[:,n])
                if n > 0 and in_stance[i, n-1]:
                    # feet should not move during stance.
                    prog.AddConstraint(partial(fixed_position_constraint, context_index=n-1, frame=foot_frame[i]),
                                       lb=np.zeros(3), ub=np.zeros(3), vars=np.concatenate((q[:,n-1], q[:,n])))
            else:
                min_clearance = 0.04
                prog.AddConstraint(PositionConstraint(plant, plant.world_frame(), [-np.inf,-np.inf,min_clearance], [np.inf,np.inf,np.inf],foot_frame[i],[0,0,0],context[n]), q[:,n])
    
    # Periodicity
    q_selector = PositionView([True]*nq)
    q_selector.body_x = False
    prog.AddLinearConstraint(eq(q[q_selector,0], q[q_selector,-1]))
    prog.AddLinearConstraint(eq(v[:,0], v[:,-1]))

    # TODO: Set solver parameters (mostly to make the worst case solve times less bad)
    snopt = SnoptSolver().solver_id()
    prog.SetSolverOption(snopt, 'Iterations Limits', 1e7 if running_as_notebook else 1)
    prog.SetSolverOption(snopt, 'Major Iterations Limit', 1000 if running_as_notebook else 1)
    prog.SetSolverOption(snopt, 'Major Feasibility Tolerance', 5e-6)
    prog.SetSolverOption(snopt, 'Major Optimality Tolerance', 1e-4)
    prog.SetSolverOption(snopt, 'Superbasics limit', 2000)
    prog.SetSolverOption(snopt, 'Linesearch tolerance', 0.9)
    #prog.SetSolverOption(snopt, 'Print file', 'snopt.out')

    result = Solve(prog)
    print(result)
    print(result.get_solver_id().name())
    print(result.is_success())
    
    # Animate trajectory
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    t_sol = np.cumsum(np.concatenate(([0],result.GetSolution(h))))
    q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
    visualizer.start_recording()
    num_strides = 2
    
    t0 = t_sol[0]
    tf = t_sol[-1]
    T = tf*num_strides
    for _ in range(10):
        for t in np.hstack((np.arange(t0, T, visualizer.draw_period), T)):
            context.SetTime(t)
            stride = (t - t0) // (tf - t0)
            ts = (t - t0) % (tf - t0)
            qt = PositionView(q_sol.value(ts))
            qt.body_x += stride*stride_length
            plant.SetPositions(plant_context, qt)
            diagram.Publish(context)

    visualizer.stop_recording()
    visualizer.publish_recording()

    print(np.min(result.GetSolution(q)[-2,:]))
    print(np.max(result.GetSolution(q)[-2,:]))
    print(np.min(result.GetSolution(q)[-1,:]))
    print(np.max(result.GetSolution(q)[-1,:]))

#gaitoptimization(gait="walking")
gaitoptimization(gait="walking")

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://371807718113.ngrok.io/static/
Connected to meshcat-server.
---------------------------------------------------------------
Starting state:
[qw, qx, qy, qz, x, y, z, frontjointangle, backjointangle]
Note: negative angle means tilting the leg forwards
[1.  0.  0.  0.  0.  0.  0.1 0.  0. ]
Total mass:
1.8928
Gravity vector:
[ 0.    0.   -9.81]
nq: 9
nv: 8
SNOPT/fortran
True
-0.9440452513232036
0.5690559745641016
0.0015413005027758275
1.6263023372877237


In [ ]:
# Def terrain object
class Region(object):
    def __init__(self, bounds=[-np.inf, np.inf, -np.inf, np.inf, 1e-3, np.inf], is_contact=False):
        """
        Define a region.
        bounds: list - [lower x bound, upper x bound, lower y bound, upper y bound, lower z bound, upper z bound]
        is_contact: bool - True if region is a contact region, false for freespace.
        """
        self.bounds = bounds
        self.is_contact = is_contact
        self.names = ["lower_x_bound", "upper_x_bound", "lower_y_bound", "upper_y_bound", "lower_z_bound", "upper_z_bound"]
        for prefix, suffix in zip(self.names, self.bounds):
            setattr(self, prefix, suffix)
    
    def list_info(self):
        print("Contact region:", str(self.is_contact)) 
        for prefix, suffix in zip(self.names, self.bounds):
            print(prefix + ":", str(suffix))
    
    def lower_bounds(self):
        return np.array([self.lower_x_bound, self.lower_y_bound, self.lower_z_bound])
    
    def upper_bounds(self):
        return np.array([self.upper_x_bound, self.upper_y_bound, self.upper_z_bound])

class Terrain(object):
    def __init__(self, regions=[]):
        self.regions = regions
    
    def num_regions(self):
        return len(self.regions)

    def add_region(self, region):
        self.regions.append(region)

    def list_regions(self):
        print("Terrain Regions")
        print("----------------------------------------------")
        for reg in self.regions:
            reg.list_info()
            print("----------------------------------------------")

In [ ]:
terrain = Terrain([Region()])
contact_region = Region(bounds=[-np.inf, np.inf, -np.inf, np.inf, 0, 0], is_contact=True)
terrain.add_region(contact_region)
terrain.list_regions()

Terrain Regions
----------------------------------------------
Contact region: False
lower_x_bound: -inf
upper_x_bound: inf
lower_y_bound: -inf
upper_y_bound: inf
lower_z_bound: 0.001
upper_z_bound: inf
----------------------------------------------
Contact region: True
lower_x_bound: -inf
upper_x_bound: inf
lower_y_bound: -inf
upper_y_bound: inf
lower_z_bound: 0
upper_z_bound: 0
----------------------------------------------


In [ ]:
def add_mccormick_envelope_constraint(add, u, v):
    pass

def miqp_optimization(terrain):
    # Set up simulated environment
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    littledog = parser.AddModelFromFile('littledog/LittleDog3.urdf')
    plant.Finalize()
    visualizer = ConnectMeshcatVisualizer(builder, 
        scene_graph=scene_graph, 
        zmq_url=zmq_url)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    set_home(plant, plant_context) # Set robot to default position
    visualizer.load()
    diagram.Publish(context)

    q0 = plant.GetPositions(plant_context)
    body_frame = plant.GetFrameByName("body")

    PositionView = MakeNamedViewPositions(plant, "Positions")
    VelocityView = MakeNamedViewVelocities(plant, "Velocities")

    mu = 1 # rubber on rubber

    # Compute
    total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(littledog))
    gravity = plant.gravity_field().gravity_vector()
    
    print("---------------------------------------------------------------")
    print("Starting state:")
    print("[qw, qx, qy, qz, x, y, z, frontjointangle, backjointangle]")
    print("Note: negative angle means tilting the leg forwards")
    print(q0)
    print("Total mass:")
    print(total_mass)
    print("Gravity vector:")
    print(gravity)
    print("---------------------------------------------------------------")
    terrain.list_regions()

    foot_frame = [
        plant.GetFrameByName('front_right_foot_center'),
        plant.GetFrameByName('back_right_foot_center')]
    
    """
    TODO: Setup Gait Stuff
    """
    # Below values are subject to change. They are used for walking_trot
    N = 21
    speed = 0.4
    stride_length = 0.2 # Original was .25, but legs are shorter now
    
    T = stride_length / speed

    # Initialize Optimization
    prog = MathematicalProgram()

    # Time steps (uniform)   
    h = np.ones(N-1) * (T/(N-1))
    
    # Create one context per timestep (to maximize cache hits)
    context = [plant.CreateDefaultContext() for i in range(N)]
    # We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
    ad_plant = plant.ToAutoDiffXd()

   
    
    # Contact forces
    num_feet = 2
    
    # Tells which feet are in which region and when
    # Create list with num_feet sets of decision variables, each corresponding to a foot
    # in_region[foot_index][r, n] = 1 <==> Foot is in region r at time n
    in_region = [prog.NewBinaryVariables(rows=terrain.num_regions(), cols=N) for _ in range(num_feet)]
    # Ensure foot is only in one region at any timestep
    for i in range(num_feet):
        for n in range(N):
            prog.AddLinearConstraint(eq(np.sum(in_region[i][:, n]), [1]))
    prog.AddQuadraticErrorCost(np.array([0]), np.array([0]), np.array([in_region[0][0,0]]))
    bb = branch_and_bound.MixedIntegerBranchAndBound(prog, OsqpSolver().solver_id())
    # result = bb.Solve()
    
    # # print(result.get_solver_id().name())
    # print(result.name)
    # print(result.value)
    cost = bb.GetOptimalCost()
    result = bb.GetSolution(in_region[0][0,0])
    """
    # Enforce dynamics: friction & contact forces
    contact_force = [prog.NewContinuousVariables(3, N-1, f"foot{foot}_contact_force") for foot in range(num_feet)]
    for n in range(N-1):
        for foot in range(num_feet):
            # Linear friction cone
            prog.AddLinearConstraint(contact_force[foot][0,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(-contact_force[foot][0,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(contact_force[foot][1,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(-contact_force[foot][1,n] <= mu*contact_force[foot][2,n])
            # Each leg is either in a contact or non-contact region.
            # If in contact: The region it's in will give bounds [0, (1-1)*M + M = M], all other regions will give constraints [0, M + M (or + 0 if not contact)] <-- doesn't constrain it further
            # If not in contact: The region it's in will give bounds [0, (1-1)*M + 0*M = 0], all other regions will give constraints [0, M + M (or + 0 if not contact)] <-- doesn't constrain it further
            big_M = 4*9.81*total_mass
            prog.AddLinearConstraint(contact_force[foot][2,n] <= big_M)
            for t in range(terrain.num_regions()):
                # Note: may need to add [] around some stuff
                reg = terrain.regions[t]
                if not reg.is_contact:
                    prog.AddLinearConstraint(contact_force[foot][2,n] <= (1-in_region[foot][t,n])*big_M)
                prog.AddLinearConstraint(0 <= contact_force[foot][2,n])

    # Enforce dynamics: center of mass
    # Center of mass variables and constraints
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N-1, "comddot")
    # Initial CoM x,y position == 0
    prog.AddBoundingBoxConstraint(0, 0, com[:2,0])
    # Initial CoM z vel == 0
    prog.AddBoundingBoxConstraint(0, 0, comdot[2,0])
    # CoM height
    prog.AddBoundingBoxConstraint(.08, np.inf, com[2,:])
    # CoM x velocity >= 0
    prog.AddBoundingBoxConstraint(0, np.inf, comdot[0,:])
    # CoM final x position
    prog.AddBoundingBoxConstraint(stride_length, stride_length, com[0,-1])
    # CoM dynamics
    for n in range(N-1):
        prog.AddConstraint(eq(com[:, n+1], com[:,n] + h[n]*comdot[:,n]))
        prog.AddConstraint(eq(comdot[:, n+1], comdot[:,n] + h[n]*comddot[:,n]))
        prog.AddConstraint(eq(total_mass*comddot[:,n], sum(contact_force[i][:,n] for i in range(num_feet)) + total_mass*gravity))    
    
    feet_position = [prog.NewContinuousVariables(3, N, f"foot{foot}_position") for foot in range(num_feet)] # Foot position w.r.t. body COM
    for n in range(N):
        # Make sure foot is in bounding box relative to body frame.
        # x in +- [0.03, .125], z in [-.099, -.073]
        upper = [0.125, 0, -0.0737]
        lower = [0.03, 0, -0.099]
        for i in range(3):
            prog.AddLinearConstraint(feet_position[0][i, n] <= upper[i])
            prog.AddLinearConstraint(feet_position[0][i, n] >= lower[i])
        
        #prog.AddLinearConstraint(feet_position[1][:, n] <= np.array([[-0.03, 0, -0.0737]]).T)
        #prog.AddLinearConstraint(feet_position[1][:, n] >= np.array([[-0.125, 0, -0.099]]).T)
    

    

    
    
    # Animate trajectory
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    t_sol = np.cumsum(np.concatenate(([0],result.GetSolution(h))))
    q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
    visualizer.start_recording()
    num_strides = 10
    
    t0 = t_sol[0]
    tf = t_sol[-1]
    T = tf*num_strides
    for t in np.hstack((np.arange(t0, T, visualizer.draw_period), T)):
        context.SetTime(t)
        stride = (t - t0) // (tf - t0)
        ts = (t - t0) % (tf - t0)
        qt = PositionView(q_sol.value(ts))
        qt.body_x += stride*stride_length
        plant.SetPositions(plant_context, qt)
        diagram.Publish(context)

    visualizer.stop_recording()
    visualizer.publish_recording()
    """
miqp_optimization(terrain)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://371807718113.ngrok.io/static/
Connected to meshcat-server.
---------------------------------------------------------------
Starting state:
[qw, qx, qy, qz, x, y, z, frontjointangle, backjointangle]
Note: negative angle means tilting the leg forwards
[1.  0.  0.  0.  0.  0.  0.1 0.  0. ]
Total mass:
1.8928
Gravity vector:
[ 0.    0.   -9.81]
---------------------------------------------------------------
Terrain Regions
----------------------------------------------
Contact region: False
lower_x_bound: -inf
upper_x_bound: inf
lower_y_bound: -inf
upper_y_bound: inf
lower_z_bound: 0.001
upper_z_bound: inf
----------------------------------------------
Contact region: True
lower_x_bound: -inf
upper_x_bound: inf
lower_y_bound: -inf
upper_y_bound: inf
lower_z_bound: 0
upper_z_bound: 0
----------------------------------------------


RuntimeError: ignored

In [ ]:
from functools import partial
from pydrake.all import (
    MultibodyPlant, JointIndex, RotationMatrix, PiecewisePolynomial, JacobianWrtVariable,
    MathematicalProgram, Solve, eq, AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
)
from pydrake.common.containers import namedview

# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(autoDiffToGradientMatrix(a), autoDiffToGradientMatrix(b))

# TODO: promote this to drake (and make a version with model_instance)
def MakeNamedViewPositions(mbp, view_name):
    names = [None]*mbp.num_positions()
    for ind in range(mbp.num_joints()): 
        joint = mbp.get_joint(JointIndex(ind))
        # TODO: Handle planar joints, etc.
        assert(joint.num_positions() == 1)
        names[joint.position_start()] = joint.name()
    for ind in mbp.GetFloatingBaseBodies():
        body = mbp.get_body(ind)
        start = body.floating_positions_start()
        body_name = body.name()
        names[start] = body_name+'_qw'
        names[start+1] = body_name+'_qx'
        names[start+2] = body_name+'_qy'
        names[start+3] = body_name+'_qz'
        names[start+4] = body_name+'_x'
        names[start+5] = body_name+'_y'
        names[start+6] = body_name+'_z'
    return namedview(view_name, names)

def MakeNamedViewVelocities(mbp, view_name):
    names = [None]*mbp.num_velocities()
    for ind in range(mbp.num_joints()): 
        joint = mbp.get_joint(JointIndex(ind))
        # TODO: Handle planar joints, etc.
        assert(joint.num_velocities() == 1)
        names[joint.velocity_start()] = joint.name()
    for ind in mbp.GetFloatingBaseBodies():
        body = mbp.get_body(ind)
        start = body.floating_velocities_start() - mbp.num_positions()
        body_name = body.name()
        names[start] = body_name+'_wx'
        names[start+1] = body_name+'_wy'
        names[start+2] = body_name+'_wz'
        names[start+3] = body_name+'_vx'
        names[start+4] = body_name+'_vy'
        names[start+5] = body_name+'_vz'
    return namedview(view_name, names)

def set_home(plant, context):
    knee = 0 #.5
    starting_height = .1 #.1 should be exactly on ground

    plant.GetJointByName("front_right_knee").set_angle(context, -knee)
    
    plant.GetJointByName("back_right_knee").set_angle(context, knee)
    plant.SetFreeBodyPose(context, plant.GetBodyByName("body"), RigidTransform([0, 0, starting_height]))


def gaitoptimization(terrain, gait="walking"):
    # Set up simulated environment
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 1e-3)
    parser = Parser(plant)
    littledog = parser.AddModelFromFile('littledog/LittleDog3.urdf')
    plant.Finalize()
    visualizer = ConnectMeshcatVisualizer(builder, 
        scene_graph=scene_graph, 
        zmq_url=zmq_url)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    set_home(plant, plant_context) # Set robot to default position
    visualizer.load()
    diagram.Publish(context)

    q0 = plant.GetPositions(plant_context)
    body_frame = plant.GetFrameByName("body")

    PositionView = MakeNamedViewPositions(plant, "Positions")
    VelocityView = MakeNamedViewVelocities(plant, "Velocities")

    mu = 1 # rubber on rubber

    # Compute
    total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(littledog))
    gravity = plant.gravity_field().gravity_vector()
    
    print("---------------------------------------------------------------")
    print("Starting state:")
    print("[qw, qx, qy, qz, x, y, z, frontjointangle, backjointangle]")
    print("Note: negative angle means tilting the leg forwards")
    print(q0)
    print("Total mass:")
    print(total_mass)
    print("Gravity vector:")
    print(gravity)

    foot_frame = [
        plant.GetFrameByName('front_right_foot_center'),
        plant.GetFrameByName('back_right_foot_center')]

    """
    TODO: Setup Gait Stuff
    """
    if gait == "walking":
        N = 21
        speed = 0.4
        stride_length = 0.2 # Original was .25, but legs are shorter now
        in_stance = np.zeros((2, N))
        in_stance[0, :11] = 1
        in_stance[0, -1] = 1
        in_stance[1, 0] = 1
        in_stance[1, 8:N] = 1
    elif gait == "running":
        N = 21
        speed = 0.5
        stride_length = 0.25
        in_stance = np.zeros((2, N))
        in_stance[0, :8] = 1
        in_stance[0, -1] = 1
        in_stance[1, 0] = 1
        in_stance[1, 13:N] = 1
    else:
        print("Unknown gait")
        return
    
    T = stride_length / speed

    # Initialize Optimization
    prog = MathematicalProgram()

    # Time steps    
    h = prog.NewContinuousVariables(N-1, "h")
    prog.AddBoundingBoxConstraint(0.5*T/N, 2.0*T/N, h) 
    prog.AddLinearConstraint(sum(h) >= .9*T)
    prog.AddLinearConstraint(sum(h) <= 1.1*T)

    
    num_feet = 2
    
    # Tells which feet are in which region and when
    # Create list with num_feet sets of decision variables, each corresponding to a foot
    # in_region[foot_index][r, n] = 1 <==> Foot is in region r at time n
    in_region = [prog.NewBinaryVariables(rows=terrain.num_regions(), cols=N) for _ in range(num_feet)]    
    
    
    # Create one context per timestep (to maximize cache hits)
    context = [plant.CreateDefaultContext() for i in range(N)]
    # We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
    ad_plant = plant.ToAutoDiffXd()

    # Joint positions and velocities
    nq = plant.num_positions()
    nv = plant.num_velocities()
    q = prog.NewContinuousVariables(nq, N, "q")
    v = prog.NewContinuousVariables(nv, N, "v")
    q_view = PositionView(q)
    v_view = VelocityView(v)
    q0_view = PositionView(q0)
    
    # Joint costs
    q_cost = PositionView([1]*nq)
    v_cost = VelocityView([1]*nv)
    
    q_cost.body_x = 0
    q_cost.body_y = 0
    q_cost.body_qx = 0
    q_cost.body_qy = 0
    q_cost.body_qz = 0
    q_cost.body_qw = 0

    v_cost.body_vx = 0
    v_cost.body_wx = 0
    v_cost.body_wy = 0
    v_cost.body_wz = 0

    print("nq:",nq) # COM quaternion orientation (4) + COM xyz position (3) + joint angles (2) = 9
    print("nv:",nv) # COM axis-angle angular velocity (3) + COM xyz velocity (3) + joint angle velocity (2) = 8

    # Formulate optimization problem
    for n in range(N):
        # Joint limits
        prog.AddBoundingBoxConstraint(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits(), q[:,n])
        # Joint velocity limits
        prog.AddBoundingBoxConstraint(plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits(), v[:,n])
        # Unit quaternions
        AddUnitQuaternionConstraintOnPlant(plant, q[:,n], prog)
        # Body orientation
        prog.AddConstraint(OrientationConstraint(plant, 
                                                 body_frame, RotationMatrix(),
                                                 plant.world_frame(), RotationMatrix(), 
                                                 0.1, context[n]), q[:,n])
        # Initial guess for all joint angles is the home position
        prog.SetInitialGuess(q[:,n], q0)  # Solvers get stuck if the quaternion is initialized with all zeros.

        # Running costs:
        prog.AddQuadraticErrorCost(np.diag(q_cost), q0, q[:,n]) # Add cost with q_desired = q0 (default)
        prog.AddQuadraticErrorCost(np.diag(v_cost), [0]*nv, v[:,n])
    
    # Make a new autodiff context for this constraint (to maximize cache hits)
    ad_velocity_dynamics_context = [ad_plant.CreateDefaultContext() for i in range(N)]
    
    def velocity_dynamics_constraint(vars, context_index):
        h, q, v, qn = np.split(vars, [1, 1+nq, 1+nq+nv])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_velocity_dynamics_context[context_index])):
                ad_plant.SetPositions(ad_velocity_dynamics_context[context_index], q)
            v_from_qdot = ad_plant.MapQDotToVelocity(ad_velocity_dynamics_context[context_index], (qn - q)/h)
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            v_from_qdot = plant.MapQDotToVelocity(context[context_index], (qn - q)/h)
        return v - v_from_qdot

    # Enforce dynamics: velocity decision variables = actual velocity
    for n in range(N-1):
        prog.AddConstraint(
            partial(velocity_dynamics_constraint, context_index=n), 
            lb=[0]*nv, ub=[0]*nv, 
            vars=np.concatenate(([h[n]], q[:,n], v[:,n], q[:,n+1])))
    
    # Contact forces
    num_feet = 2

    # Enforce dynamics: friction & contact forces
    contact_force = [prog.NewContinuousVariables(3, N-1, f"foot{foot}_contact_force") for foot in range(num_feet)]
    for n in range(N-1):
        for foot in range(num_feet):
            # Linear friction cone
            prog.AddLinearConstraint(contact_force[foot][0,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(-contact_force[foot][0,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(contact_force[foot][1,n] <= mu*contact_force[foot][2,n])
            prog.AddLinearConstraint(-contact_force[foot][1,n] <= mu*contact_force[foot][2,n])
            # normal force >=0, normal_force == 0 if not in_stance
            # ASSUMES THAT IN_STANCE IS GIVEN
            prog.AddBoundingBoxConstraint(0, in_stance[foot,n]*4*9.81*total_mass, contact_force[foot][2,n])

    # Enforce dynamics: center of mass
    # Center of mass variables and constraints
    com = prog.NewContinuousVariables(3, N, "com")
    comdot = prog.NewContinuousVariables(3, N, "comdot")
    comddot = prog.NewContinuousVariables(3, N-1, "comddot")
    # Initial CoM x,y position == 0
    prog.AddBoundingBoxConstraint(0, 0, com[:2,0])
    # Initial CoM z vel == 0
    prog.AddBoundingBoxConstraint(0, 0, comdot[2,0])
    # CoM height
    prog.AddBoundingBoxConstraint(.08, np.inf, com[2,:])
    # CoM x velocity >= 0
    prog.AddBoundingBoxConstraint(0, np.inf, comdot[0,:])
    # CoM final x position
    prog.AddBoundingBoxConstraint(stride_length, stride_length, com[0,-1])
    # CoM dynamics
    for n in range(N-1):
        prog.AddConstraint(eq(com[:, n+1], com[:,n] + h[n]*comdot[:,n]))
        prog.AddConstraint(eq(comdot[:, n+1], comdot[:,n] + h[n]*comddot[:,n]))
        prog.AddConstraint(eq(total_mass*comddot[:,n], sum(contact_force[i][:,n] for i in range(num_feet)) + total_mass*gravity))    
    
    # Angular momentum (about the center of mass)
    H = prog.NewContinuousVariables(3, N, "H")
    Hdot = prog.NewContinuousVariables(3, N-1, "Hdot")
    prog.SetInitialGuess(H, np.zeros((3, N)))
    prog.SetInitialGuess(Hdot, np.zeros((3,N-1)))
    # Hdot = sum_i cross(p_FootiW-com, contact_force_i)
    
    def angular_momentum_constraint(vars, context_index):
        q, com, Hdot, contact_force = np.split(vars, [nq, nq+3, nq+6])
        contact_force = contact_force.reshape(3, num_feet, order='F')
        if isinstance(vars[0], AutoDiffXd):
            q = autoDiffToValueMatrix(q)
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_feet):
                p_FW = plant.CalcPointsPositions(context[context_index], foot_frame[i], [0,0,0], plant.world_frame())
                Jq_FW = plant.CalcJacobianTranslationalVelocity(
                    context[context_index], JacobianWrtVariable.kQDot,
                    foot_frame[i], [0, 0, 0], plant.world_frame(), plant.world_frame())
                ad_p_FW = initializeAutoDiffGivenGradientMatrix(p_FW, np.hstack((Jq_FW, np.zeros((3, 18)))))
                torque = torque     + np.cross(ad_p_FW.reshape(3) - com, contact_force[:,i])
        else:
            if not np.array_equal(q, plant.GetPositions(context[context_index])):
                plant.SetPositions(context[context_index], q)
            torque = np.zeros(3)
            for i in range(num_feet):
                p_FW = plant.CalcPointsPositions(context[context_index], foot_frame[i], [0,0,0], plant.world_frame())
                torque += np.cross(p_FW.reshape(3) - com, contact_force[:,i])
        return Hdot - torque
    
    

    # Enforce dynamics: torque decision variables = actual torque
    for n in range(N-1):
        prog.AddConstraint(eq(H[:,n+1], H[:,n] + h[n]*Hdot[:,n]))
        Fn = np.concatenate([contact_force[i][:,n] for i in range(num_feet)])
        prog.AddConstraint(partial(angular_momentum_constraint, context_index=n), lb=np.zeros(3), ub=np.zeros(3), 
                           vars=np.concatenate((q[:,n], com[:,n], Hdot[:,n], Fn)))
    
    
    # Make a new autodiff context for this constraint (to maximize cache hits)
    com_constraint_context = [ad_plant.CreateDefaultContext() for i in range(N)]
    def com_constraint(vars, context_index):
        qv, com, H = np.split(vars, [nq+nv, nq+nv+3])
        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(qv, ad_plant.GetPositionsAndVelocities(com_constraint_context[context_index])):
                ad_plant.SetPositionsAndVelocities(com_constraint_context[context_index], qv)
            com_q = ad_plant.CalcCenterOfMassPositionInWorld(com_constraint_context[context_index])
            H_qv = ad_plant.CalcSpatialMomentumInWorldAboutPoint(com_constraint_context[context_index], com).rotational()
        else:
            if not np.array_equal(qv, plant.GetPositionsAndVelocities(context[context_index])):
                plant.SetPositionsAndVelocities(context[context_index], qv)
            com_q = plant.CalcCenterOfMassPositionInWorld(context[context_index])
            H_qv = plant.CalcSpatialMomentumInWorldAboutPoint(context[context_index], com).rotational()
        return np.concatenate((com_q - com, H_qv - H))
    
    for n in range(N):
        tempvars = np.concatenate((q[:,n], v[:,n], com[:,n], H[:,n]))
        prog.AddConstraint(partial(com_constraint, context_index=n), 
            lb=np.zeros(6), ub=np.zeros(6), vars=tempvars)
        
    
         
    # Kinematic constraints
    def fixed_position_constraint(vars, context_index, frame):
        q, qn = np.split(vars, [nq])
        if not np.array_equal(q, plant.GetPositions(context[context_index])):
            plant.SetPositions(context[context_index], q)
        if not np.array_equal(qn, plant.GetPositions(context[context_index+1])):
            plant.SetPositions(context[context_index+1], qn)
        p_AQ = plant.CalcPointsPositions(context[context_index], frame, [0,0,0], plant.world_frame())
        p_AQn = plant.CalcPointsPositions(context[context_index+1], frame, [0,0,0], plant.world_frame())
        if isinstance(vars[0], AutoDiffXd):
            J = plant.CalcJacobianTranslationalVelocity(context[context_index], JacobianWrtVariable.kQDot,
                                                    frame, [0, 0, 0], plant.world_frame(), plant.world_frame())
            Jn = plant.CalcJacobianTranslationalVelocity(context[context_index+1], JacobianWrtVariable.kQDot,
                                                    frame, [0, 0, 0], plant.world_frame(), plant.world_frame())
            return initializeAutoDiffGivenGradientMatrix(
                p_AQn - p_AQ, Jn @ autoDiffToGradientMatrix(qn) - J @ autoDiffToGradientMatrix(q))
        else:
            return p_AQn - p_AQ
    
    for i in range(num_feet):
        for n in range(N):
            if in_stance[i, n]:
                # foot should be on the ground (world position z=0)
                prog.AddConstraint(PositionConstraint(
                    plant, plant.world_frame(), [-np.inf,-np.inf,0], [np.inf,np.inf,0], 
                    foot_frame[i], [0,0,0], context[n]), q[:,n])
                if n > 0 and in_stance[i, n-1]:
                    # feet should not move during stance.
                    prog.AddConstraint(partial(fixed_position_constraint, context_index=n-1, frame=foot_frame[i]),
                                       lb=np.zeros(3), ub=np.zeros(3), vars=np.concatenate((q[:,n-1], q[:,n])))
            else:
                min_clearance = 0.04
                prog.AddConstraint(PositionConstraint(plant, plant.world_frame(), [-np.inf,-np.inf,min_clearance], [np.inf,np.inf,np.inf],foot_frame[i],[0,0,0],context[n]), q[:,n])
    
    # Periodicity
    q_selector = PositionView([True]*nq)
    q_selector.body_x = False
    prog.AddLinearConstraint(eq(q[q_selector,0], q[q_selector,-1]))
    prog.AddLinearConstraint(eq(v[:,0], v[:,-1]))

    # TODO: Set solver parameters (mostly to make the worst case solve times less bad)
    snopt = SnoptSolver().solver_id()
    prog.SetSolverOption(snopt, 'Iterations Limits', 1e7 if running_as_notebook else 1)
    prog.SetSolverOption(snopt, 'Major Iterations Limit', 1000 if running_as_notebook else 1)
    prog.SetSolverOption(snopt, 'Major Feasibility Tolerance', 5e-6)
    prog.SetSolverOption(snopt, 'Major Optimality Tolerance', 1e-4)
    prog.SetSolverOption(snopt, 'Superbasics limit', 2000)
    prog.SetSolverOption(snopt, 'Linesearch tolerance', 0.9)
    #prog.SetSolverOption(snopt, 'Print file', 'snopt.out')

    bb = branch_and_bound.MixedIntegerBranchAndBound(prog, OsqpSolver().solver_id())
    # result = bb.Solve()
    
    # # print(result.get_solver_id().name())
    # print(result.name)
    # print(result.value)
    cost = bb.GetOptimalCost()
    result = bb.GetSolution(in_region[0][0,0])
    
    print(result)
    print(result.get_solver_id().name())
    print(result.is_success())
    
    # Animate trajectory
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    t_sol = np.cumsum(np.concatenate(([0],result.GetSolution(h))))
    q_sol = PiecewisePolynomial.FirstOrderHold(t_sol, result.GetSolution(q))
    visualizer.start_recording()
    num_strides = 2
    
    t0 = t_sol[0]
    tf = t_sol[-1]
    T = tf*num_strides
    for _ in range(10):
        for t in np.hstack((np.arange(t0, T, visualizer.draw_period), T)):
            context.SetTime(t)
            stride = (t - t0) // (tf - t0)
            ts = (t - t0) % (tf - t0)
            qt = PositionView(q_sol.value(ts))
            qt.body_x += stride*stride_length
            plant.SetPositions(plant_context, qt)
            diagram.Publish(context)

    visualizer.stop_recording()
    visualizer.publish_recording()

    print(np.min(result.GetSolution(q)[-2,:]))
    print(np.max(result.GetSolution(q)[-2,:]))
    print(np.min(result.GetSolution(q)[-1,:]))
    print(np.max(result.GetSolution(q)[-1,:]))

#gaitoptimization(gait="walking")
gaitoptimization(terrain, gait="running")